In [2]:
from transformers import BartForConditionalGeneration, AutoTokenizer
bart = BartForConditionalGeneration.from_pretrained('hyunwoongko/kobart')
tokenizer = AutoTokenizer.from_pretrained('hyunwoongko/kobart')

In [3]:
texts = ['최근 들어 딥러닝', '안녕 친구야']
sample_ids = tokenizer(texts, padding=True, return_tensors='pt')['input_ids']
sample_ids

tensor([[19261, 14301,  1700, 10021, 10277,  9747],
        [11699,  9592, 15230, 11734,     3,     3]])

In [4]:
outs = bart.generate(
    sample_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
    early_stopping=True,
)